# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** R
**Names:**

* Raphael Strebel
* Raphaël Barman
* Thierry Bossy

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [28]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl
import string
import re
import operator
import nltk
import math

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Exercise 4.1: Pre-processing

In [3]:
lmtzr = WordNetLemmatizer()
stemmer = PorterStemmer()

def add2bow(word, bow):
    if word not in bow:
        bow[word] = 0
    bow[word] += 1

def mergeBow(bow1, bow2):
    for word, occ in bow2.items():
        if word not in bow1:
            bow1[word] = 0
        bow1[word] += occ

def bagOfWord(text):
    bow = {}
    text = text.replace('\xa0', ' ')
    # Remove punctuation
    text = text.translate(str.maketrans('','',string.punctuation))
    text = text.split(' ')
    for idx in range(len(text)):
        word = text[idx]
        # separate words such that "MyNameIsChristian" becomes "My" "Name" "Is" "Christian"
        res = re.findall('[a-zA-Z][^A-Z]*',word)
        if res:
            if len(min(res,key=len)) != 1:
                if len(res) > 0:
                    text[idx] = ''
                for match in res:
                    text.append(match)
    text = [x for x in text if x != '']
    for idx in range(len(text)):
        word = text[idx]
        # Keep words that are only upper case as such (we don't want IT to become it) and put all others as lower case
        if not word.isupper():
            word = word.lower()
        # Lemmatize all non-digit words 
        if not word.isdigit() and not word in stopwords:
            add2bow(lmtzr.lemmatize(word),bow)
            #add2bow(stemmer.stem(word),bow)
#     for word in textCopy:
#         if not word.isupper():
#             word = word.lower()
#         if not word.isdigit():
#             text.add(lmtzr.lemmatize(word))
#     for word in stopwords:
#         try:
#             text.remove(word)
#         except KeyError:
#             continue
    return bow

In [4]:
bagOfWords = {}
globalBagOfWord = {}
bagOfWord(courses[1]['description'])
for course in courses:
    bow = bagOfWord(course['description'])
    bagOfWords[course['courseId']] = bow
    mergeBow(globalBagOfWord,bow)
test_course = course.copy()

We chose to remove all punctuation and all stopwords since there really is no interest in keeping them.
We also lemmatize the words using the nltk library, to keep track of similar words and have a more accurate word occurence count. 

In [5]:
test_course = courses[1]
bagOfWords = {}
#globalBagOfWord = {}
#bagOfWord(courses[1]['description'])
print(test_course)
bow = bagOfWord(test_course['description'])
bagOfWords[test_course['courseId']] = bow
#mergeBow(globalBagOfWord,bow)
print(bow)
dict(sorted(bagOfWords.items(), key=operator.itemgetter(1), reverse=True)[:5])


{'name': 'Image Processing for Life Science', 'description': 'This course intends to teach image processing with a strong emphasis of applications in life sciences. The idea is to enable the participants to solve image processing questions via workflows independently. Content Over the last decades, the images arising from microscopes in Life Sciences went from being a qualitative support of scientific evidence to a quantitative resource. To obtain good quality data from digital images, be it from a photograph of a Western blot, a TEM slice or a multi-channel confocal time-lapse stack, scientists must understand the underlying processes leading to the extracted information. Of similar importance is the software used to obtain the data. This course makes use of the ImageJ (FIJI package) as well as other open-source tools to ensure maximum reproducibility and protocol transfer of the analysis pipelines. The course will span 14 weeks with 1h30 of lecture per week, as well as exercises to c

{'BIO-695': {'FIJI': 2,
  'TEM': 1,
  'aim': 1,
  'analysis': 3,
  'application': 1,
  'arising': 1,
  'assessment': 1,
  'autonomous': 1,
  'autonomously': 1,
  'biology': 1,
  'blot': 1,
  'complete': 1,
  'completed': 1,
  'confocal': 1,
  'content': 1,
  'context': 1,
  'continuous': 1,
  'cover': 1,
  'creation': 2,
  'data': 5,
  'decade': 1,
  'deconvolution': 1,
  'defined': 1,
  'denoising': 1,
  'digital': 3,
  'emphasis': 2,
  'enable': 2,
  'ensure': 1,
  'establish': 1,
  'evidence': 1,
  'exercise': 4,
  'extracted': 1,
  'extraction': 1,
  'filtering': 2,
  'format': 1,
  'goal': 1,
  'good': 2,
  'h30': 1,
  'handed': 1,
  'homework': 1,
  'hour': 1,
  'idea': 1,
  'image': 12,
  'imagej': 2,
  'importance': 1,
  'independently': 1,
  'information': 1,
  'intends': 1,
  'interest': 1,
  'introduce': 1,
  'involve': 1,
  'keywords': 1,
  'leading': 1,
  'learning': 1,
  'lecture': 2,
  'life': 3,
  'linear': 1,
  'machine': 1,
  'macro': 3,
  'make': 1,
  'manipulation':

In [6]:
min(globalBagOfWord,key=globalBagOfWord.get)
dict(sorted(globalBagOfWord.items(), key=operator.itemgetter(1), reverse=False)[:5000])
# Maybe discard only the 3 most used words? since system and design are more specific than learning, student and system

{'interconnect': 3,
 'postprimary': 3,
 'inaction': 3,
 'others6': 3,
 'mechanosensory': 3,
 'hyperelliptic': 3,
 'FDD': 3,
 'WINGS': 3,
 'FRP': 3,
 'valentine': 3,
 'curable': 3,
 'epifourier': 3,
 'applications10': 3,
 'emblematic': 3,
 'microdispersed': 3,
 'propagator': 3,
 'photopolymers': 3,
 'equivallent': 3,
 'adaption': 3,
 'aidistributed': 3,
 'techniquesformulate': 3,
 'gyromagnetic': 3,
 'perl': 3,
 'brief': 3,
 'refreshment': 3,
 'poroelasticity': 3,
 'metaloxide': 3,
 'ah20identify': 3,
 'kirchoffs': 3,
 'lawson': 3,
 'EE332': 3,
 'BBC': 3,
 'skundin': 3,
 'oksana': 3,
 'decorrelation': 3,
 'FINAL': 3,
 'NEM': 3,
 'transcriptase': 3,
 '2B': 3,
 'processmicrostructure': 3,
 'pip3signaling': 3,
 'sectoral': 3,
 'nozzle': 3,
 'vestibular': 3,
 'EPMAWDX': 3,
 'light42': 3,
 'dijkstras': 3,
 'T3': 3,
 'spontenaous': 3,
 'URL': 3,
 'multitemporal': 3,
 'principles3': 3,
 'hamming': 3,
 'httpvectorcom': 3,
 'orthopaedic': 3,
 'subunit': 3,
 'materialsapplications': 3,
 'firouzeh

In [7]:
#courses
globalBagOfWord

{'shaping3': 3,
 'mobile': 129,
 'unstructured': 6,
 'TRIZ': 6,
 'matanya': 6,
 'morphology': 39,
 'microdispersed': 3,
 'review': 384,
 'photopolymers': 3,
 'multiconductor': 12,
 'adaption': 3,
 'cryptographic': 30,
 'wired': 12,
 'gyromagnetic': 3,
 'instruction': 57,
 'supernova': 6,
 'brief': 3,
 'refreshment': 3,
 'EELS': 18,
 'ah20identify': 3,
 'kirchoffs': 3,
 'LWR': 6,
 'EE332': 3,
 'ETHZ': 9,
 'quantification': 42,
 'ito': 33,
 'mesurands': 3,
 'gu': 18,
 'sectoral': 3,
 'littlebig': 3,
 'staining': 12,
 'bench': 12,
 'crop': 9,
 'multitemporal': 3,
 'regression': 204,
 'microbiology': 30,
 'subunit': 3,
 'focal': 24,
 'vivian': 3,
 'immune': 42,
 'dilution': 6,
 'thickeningstabilisationdewatering': 3,
 'kelvin': 6,
 'reception': 3,
 'stimulation': 33,
 'twodimensionnal': 3,
 'fedbatches': 6,
 'qualtiy': 3,
 'curable': 3,
 'JW': 9,
 'CP14': 3,
 'incompatibility': 3,
 'martingale': 60,
 'siemens': 9,
 'nonnormal': 3,
 'specialisation': 12,
 'leak': 3,
 'general': 717,
 'blief

In [8]:
# Term Document Matrix
# We want a matrix where each row i is a word (among global bag of words) 
# and each column j is a document (among all courses)
# tdm[i][j] = nb of occurences of term i in doc j

# total number of terms
M = len(globalBagOfWord)

# total number of documents
N = len(courses)

termDocMatrix = np.zeros((M,N), dtype=np.int)

i = 0

# Column index
docIndx = 0

for doc in courses: 
    bow = bagOfWord(doc['description'])
    # Row index
    termIndx = 0
    for word in globalBagOfWord.keys():
        termDocMatrix[termIndx][docIndx] += bow.get(word, 0)
        termIndx += 1
    docIndx += 1

In [9]:
# Compute fij (frequency of term i in doc j), à transformer en fonction?

totalWords = len(globalBagOfWord)
totalCourses = len(courses)

f = np.zeros((totalCourses,totalWords),dtype=np.double)

docIndx = 0
for doc in courses:
    test_bow = bagOfWord(doc['description'])
    wordIndx = 0
    for word in test_bow:
        f[wordIndx][docIndx] = test_bow.get(word) / len(test_bow) 
        wordIndx += 1
    docIndx += 1

In [36]:
# Compute TFij "importance of word i in doc j"
TF = np.zeros((totalCourses,totalWords),dtype=np.double)
maxWordOfDoc = [""]*totalCourses

# Find the number of occurences of the most used words in every document
docIndx = 0
for doc in courses:
    test_bow = bagOfWord(doc['description'])
    maxWordOfDoc[docIndx] = list(test_bow.values())[np.argmax(f[docIndx])]
    docIndx += 1
    
# Compute TF
# Note: on a pas besoin d'itérer sur toutes les lignes, juste les colonnes mais j'arrive pas à utiliser 
# la fonction np.apply_along_axis en passant une fonction qui doit connaitre l'indice de la ligne
for i in range(totalWords):
    for j in range(totalCourses):
        TF[i][j] = f[i][j] / maxWordOfDoc[j]


IndexError: index 2562 is out of bounds for axis 0 with size 2562

In [31]:
# Inverse Document Frequency : IDF
# Compute n[i] = nb of documents where word i occurs at least once
n = np.zeros((totalWords),dtype=np.int)
IDF = np.zeros((totalWords),dtype=np.double)

for i in range(totalWords):
    for j in range(totalCourses):
        if(f[j][i] != 0):
            n[i] += 1
    if(n[i] == 0):
        IDF[i] = 0
    else:
        IDF[i] = -math.log2(n[i]/totalCourses)

[-0.         -0.          0.00169033 ...,  0.          0.          0.        ]


In [35]:
# Compute TF-IDF score
TFIDF = np.zeros((totalCourses,totalWords),dtype=np.double)

for i in range(totalWords):
    for j in range(totalCourses):
        TFIDF[j][i] = TF[j][i] * IDF[i]

[[-0. -0.  0. ...,  0.  0.  0.]
 [-0. -0.  0. ...,  0.  0.  0.]
 [-0. -0.  0. ...,  0.  0.  0.]
 ..., 
 [-0. -0.  0. ...,  0.  0.  0.]
 [-0. -0.  0. ...,  0.  0.  0.]
 [-0. -0.  0. ...,  0.  0.  0.]]


From the slides:

N = total number of documents

f[i][j] nb of occurrences of word 𝑖 in doc 𝑗, so bagOfWord(j)[i]

tf[i][j] = f[i][j] / max_k f[k][j]

idf[i] = -log_2(number of documents where word i occurs at least once / N)

tfidf[i][j] = tf[i][j] * idf[i]

## Exercise 4.2: Term-document matrix

## Exercise 4.3: Document similarity search